# Setup

In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F
from datetime import datetime
from collections import namedtuple
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 11:05:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
"""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 

    SUM(total_amount) AS amount,
    COUNT(*) AS number_records

FROM 
    green
WHERE
    pickup_datetime > '2020-01-01 00:00:00'
GROUP BY 
    zone, hour
"""

"\nSELECT \n    -- Reveneue grouping \n    PULocationID AS zone,\n    date_trunc('hour', pickup_datetime) AS hour, \n\n    SUM(total_amount) AS amount,\n    COUNT(*) AS number_records\n\nFROM \n    green\nWHERE\n    pickup_datetime > '2020-01-01 00:00:00'\nGROUP BY \n    zone, hour\n"

# Operations on Spark RDDs

In [7]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 20, 24, 10), PULocationID=173, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 16, 53), PULocationID=74, total_amount=23.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 14, 47, 15), PULocationID=25, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 10, 8), PULocationID=259, total_amount=25.54)]

In [12]:
start = datetime(year = 2020, month = 1, day = 1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [14]:
rows = rdd.take(10)
row = rows[0]

In [16]:
row.lpep_pickup_datetime.replace(minute = 0, second = 0, microsecond = 0)

datetime.datetime(2020, 1, 12, 18, 0)

In [18]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute = 0, second = 0, microsecond = 0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    return (key, value)

In [20]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)

In [34]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [35]:
def unwrap(row):
    return RevenueRow(hour = row[0][0], 
                      zone = row[0][1], 
                      revenue = row[1][0],
                      count = row[1][1])

In [41]:
result_schema = types.StructType(
           [types.StructField('hour', types.TimestampType(), True),
            types.StructField('zone', types.IntegerType(), True),
            types.StructField('revenue', types.DoubleType(), True),
            types.StructField('count', types.IntegerType(), True)])

In [27]:
rdd \
    .filter(filter_outliers) \
    .take(10)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 20, 24, 10), PULocationID=173, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 16, 53), PULocationID=74, total_amount=23.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 14, 47, 15), PULocationID=25, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 10, 8), PULocationID=259, total_amount=25.54),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 18, 17, 46, 45), PULocationID=177, total_amount=13.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 20, 8, 44), PULocationID=65, total_amount=11.16),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 13, 10, 47), PULocationID=165, total_amount=20.56),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 15, 36), PULocationID=170, total_amount=49.05),
 Row(lpep_pickup_datetime=datetime.datetime(202

In [28]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2020, 1, 12, 18, 0), 41), (7.88, 1)),
 ((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 74), (23.46, 1)),
 ((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1)),
 ((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1)),
 ((datetime.datetime(2020, 1, 13, 10, 0), 165), (20.56, 1)),
 ((datetime.datetime(2020, 1, 7, 15, 0), 170), (49.05, 1)),
 ((datetime.datetime(2020, 1, 10, 11, 0), 74), (9.56, 1))]

In [26]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)

[((datetime.datetime(2020, 1, 13, 10, 0), 165), (55.43000000000001, 3)),
 ((datetime.datetime(2020, 1, 23, 10, 0), 43), (315.40000000000003, 19)),
 ((datetime.datetime(2020, 1, 2, 9, 0), 116), (296.54, 17)),
 ((datetime.datetime(2020, 1, 31, 21, 0), 41), (588.1599999999999, 40)),
 ((datetime.datetime(2020, 1, 26, 0, 0), 258), (56.92, 1)),
 ((datetime.datetime(2020, 1, 11, 21, 0), 244), (336.4700000000001, 10)),
 ((datetime.datetime(2020, 1, 22, 0, 0), 22), (34.27, 1)),
 ((datetime.datetime(2020, 1, 9, 8, 0), 42), (946.8599999999991, 69)),
 ((datetime.datetime(2020, 1, 24, 1, 0), 129), (155.73000000000002, 12)),
 ((datetime.datetime(2020, 1, 12, 15, 0), 74), (1287.1899999999991, 85))]

In [42]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [44]:
df_result.write.parquet('tmp/green-revenue')

23/02/28 01:05:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


# Spark RDD mapPartition

In [6]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [14]:
rows = duration_rdd.take(10)

In [15]:
df = pd.DataFrame(rows)

In [19]:
list(df.itertuples())[0]

Pandas(Index=0, _1=2.0, _2=Timestamp('2020-01-12 18:15:04'), _3=41, _4=41, _5=0.78)

In [13]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [20]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns = columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [21]:
duration_rdd.mapPartitions(apply_model_in_batch).take(10)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-12 18:15:04'), PULocationID=41, DOLocationID=41, trip_distance=0.78, predicted_duration=3.9000000000000004),
 Pandas(Index=1, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-31 20:24:10'), PULocationID=173, DOLocationID=70, trip_distance=0.98, predicted_duration=4.9),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-07 08:16:53'), PULocationID=74, DOLocationID=236, trip_distance=2.7, predicted_duration=13.5),
 Pandas(Index=3, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-15 14:47:15'), PULocationID=25, DOLocationID=66, trip_distance=0.8, predicted_duration=4.0),
 Pandas(Index=4, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-31 10:08:00'), PULocationID=259, DOLocationID=51, trip_distance=2.33, predicted_duration=11.65),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-18 17:46:45'), PULocationID=177, DOLocationID=188, trip_distance=2.62, predicted_dura

In [25]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [26]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|3.9000000000000004|
|               4.9|
|              13.5|
|               4.0|
|             11.65|
|13.100000000000001|
|5.6499999999999995|
| 6.800000000000001|
|             55.75|
|               8.9|
|               5.0|
|             13.75|
|               5.5|
|             19.05|
|              9.25|
|              45.7|
|               5.2|
| 5.699999999999999|
|              5.75|
|4.6000000000000005|
+------------------+
only showing top 20 rows

